<a href="https://colab.research.google.com/github/sunnatillo91/ai-datascience/blob/main/05_ml_04_ML_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. MACHINE LEARNING

## 3-QADAM. Ma'lumotlarni Machine Learning uchun tayyorlash

## Pipeline (konveyer)

In [1]:
import pandas as pd
import numpy as np
import sklearn  # scikit-leaarn kutubxonasi

# Onlayn dataset joylashgan manzilni ko'rsatamiz
url = 'http://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true'
df = pd.read_csv(url)

# train
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=30)

housing= train_set.drop("median_house_value", axis=1)
housing_labels=train_set["median_house_value"].copy

housing_num=housing.drop("ocean_proximity", axis=1)

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin

#bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
  def __init__(self, add_bedrooms_per_room = True):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
    population_per_household = X[:, population_ix] / X[:, household_ix]
    if self.add_bedrooms_per_room:
      bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
      return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
    else:
      return np.c_[X, rooms_per_household, population_per_household]

# Pipeline da sonlar

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy="median")),
  ('attribs_adder', CombinedAttributesAdder()),
  ('std_scaler', StandardScaler()),
])

num_pipeline.fit_transform(housing_num)

array([[ 0.69045468, -0.74771271,  0.82109487, ..., -0.35013262,
         0.17603889,  0.37706385],
       [ 0.73042911, -0.79459048,  0.50456328, ...,  0.11173742,
        -0.01141653, -0.42460338],
       [ 0.80538117, -0.76646382, -0.36589859, ...,  0.45934877,
         0.01634478, -1.05766842],
       ...,
       [ 1.11018621, -1.20242712, -0.91982888, ...,  0.52081087,
        -0.09359537, -0.35677881],
       [ 0.68046107, -0.74302493,  1.21675935, ..., -0.66455731,
         0.0600627 ,  0.961126  ],
       [ 0.88033323, -0.69145938, -1.15722757, ...,  0.53409578,
         0.0065152 , -0.84314144]])

#FullPipeline, Pipeline da matnlar bilan ishlash

In [4]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
  ("num", num_pipeline, num_attribs),
  ("cat", OneHotEncoder(), cat_attribs),
])

In [5]:
housing_prepared = full_pipeline.fit_transform(housing)

In [6]:
housing_prepared[0:5,:]

array([[ 0.69045468, -0.74771271,  0.82109487, -0.92873698, -0.93634789,
        -0.66202411, -0.95892277, -0.50117396, -0.35013262,  0.17603889,
         0.37706385,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.73042911, -0.79459048,  0.50456328, -0.46504724, -0.56167866,
        -0.52340119, -0.56608836,  0.36500115,  0.11173742, -0.01141653,
        -0.42460338,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.80538117, -0.76646382, -0.36589859, -0.64951611, -0.85998219,
        -0.73357142, -0.82535907,  1.52370993,  0.45934877,  0.01634478,
        -1.05766842,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.24509991, -1.37587489, -0.68243018, -0.20413593, -0.31349013,
         0.30028727, -0.2806287 ,  0.11206675,  0.05881522,  0.11950524,
        -0.4395989 ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.72543231, -0.67739605,  1